In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gsvd import gsvd
import scipy.linalg

%matplotlib inline

Chargement des données

In [2]:
output_conv1 = np.load('Outputs/output_conv1.npy')
output_conv1_br = np.load('Outputs/output_conv1_br.npy')

weights = np.load('vgg16_weights.npz')

Entrées de l'algorithme

In [3]:
output = output_conv1_br.reshape(224*224,64)[:100].reshape(10,10,64)
Outputs = [output,output]

In [4]:
W_w = weights['conv1_1_W']
W_b = weights['conv1_1_b']

Cas linéaire

In [5]:
def LinearCase(Outputs, W_w, W_b, d_prime):
    k = W_w.shape[0]
    c = W_w.shape[2]
    d = W_w.shape[3]
    n = Outputs[0].shape[0]

    Y = Outputs[0].reshape(n*n,d).transpose()
    for i in range(1,len(Outputs)):
        Y = np.concatenate((Y,Outputs[i].reshape(n*n,d).transpose()),axis=1)
    Y_bar = np.mean(Y,axis=1,keepdims=True)
    Y = Y - Y_bar

    Cov = Y.dot(Y.transpose())
    w, v = np.linalg.eig(Cov)
    M = v[:d_prime].transpose().dot(v[:d_prime])
    P = Q = v[:d_prime].transpose()
    
    W = np.concatenate((W_w.reshape(k*k*c,d),W_b.reshape(1,d))).transpose()
    W_prime = Q.transpose().dot(W)
    
    b = (Y_bar - M.dot(Y_bar)).reshape(d)
    
    return P, W_prime, b

In [6]:
P, W_prime, b = LinearCase(Outputs, W_w, W_b, 50)

Cas non-linéaire

In [7]:
d_prime = 50

In [139]:
def GSVD(A,Y):
    M = np.eye(A.shape[0])
    
    U,s,V = np.linalg.svd(Y)
    sqrtYYT = U.dot(np.diag(s)).dot(U.transpose())
    
    A_tilde = M.dot(A).dot(sqrtYYT)
    U,s,V = np.linalg.svd(A_tilde,full_matrices=True)
    
    return M.dot(U), s, np.linalg.inv(sqrtYYT).dot(V.transpose())

In [140]:
k = W_w.shape[0]
c = W_w.shape[2]
d = W_w.shape[3]
n = Outputs[0].shape[0]

Y = Outputs[0].reshape(n*n,d).transpose()
for i in range(1,len(Outputs)):
    Y = np.concatenate((Y,Outputs[i].reshape(n*n,d).transpose()),axis=1)
Y_bar = np.mean(Y,axis=1,keepdims=True)
Y = Y - Y_bar

Z = np.zeros(Y.shape)

In [141]:
lambd = 0.01
for it in range(50):
    # 25 iterationq with 0.01, 25 with 1 (like in Zhang et al.)
    if it > 24:
        lambd=1
    
    # Minimization on M, b

    Z_bar = np.mean(Z,axis=1,keepdims=True)
    M_hat = Z.dot(Y.transpose()).dot(np.linalg.inv(Y.dot(Y.transpose())))
    U,s,V = GSVD(M_hat,Y)
    
    M = U.T[:d_prime].T.dot(np.diag(s[:d_prime])).dot(V.T[:d_prime])
    b = (Z_bar - M.dot(Y_bar)).reshape(d)

    # Minimization on Z

    Y_prime = M.dot(Y) + b.reshape(64,1)
    Z_0 = np.minimum(Y_prime,0)
    Z_1 = np.maximum((lambd*Y_prime + np.maximum(Y,0))/(1+lambd),0)

    for i in range(d):
        for j in range(n*n*len(Outputs)):
            q0 = (np.max(Y[i][j],0)-np.max(Z_0[i][j],0))**2 + lambd*(Z_0[i][j]-Y_prime[i][j])**2
            q1 = (np.max(Y[i][j],0)-np.max(Z_1[i][j],0))**2 + lambd*(Z_1[i][j]-Y_prime[i][j])**2
            if q0 < q1:
                Z[i][j] = Z_0[i][j]
            else:
                Z[i][j] = Z_1[i][j]
                
    print("Objective :",np.sum((np.maximum(Y,0) - np.maximum(M.dot(Y) + b.reshape(64,1),0))**2))

Objective : 1186276.10036
Objective : 554940070089.0
Objective : 1.36434839288e+15
Objective : 3.25184621715e+18
Objective : 7.74425585371e+21
Objective : 1.84425985059e+25
Objective : 4.3920207355e+28
Objective : 1.04593970309e+32
Objective : 2.49085768993e+35
Objective : 5.93186396228e+38
Objective : 1.41264634304e+42
Objective : 3.36415282479e+45
Objective : 8.01157648855e+48
Objective : 1.90792039407e+52
Objective : 4.54362538424e+55
Objective : 1.08204365845e+59
Objective : 2.57683761265e+62
Objective : 6.13662122608e+65
Objective : 1.46140835136e+69
Objective : 3.48027732322e+72
Objective : 8.28812168428e+75


KeyboardInterrupt: 

TESTS : Fonction GSVD

In [157]:
A = M_hat
M = np.eye(A.shape[0])

U_Y,s_Y,V_Y = np.linalg.svd(Y)
print("Test SVD :",np.linalg.norm((U_Y.transpose().dot(U_Y)) - np.eye(d),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((U_Y.dot(U_Y.transpose())) - np.eye(d),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((V_Y.transpose().dot(V_Y)) - np.eye(Y.shape[1]),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((V_Y.dot(V_Y.transpose())) - np.eye(Y.shape[1]),1),'(should be 0)')

YYT = U_Y.dot(np.diag(s_Y)**2).dot(U_Y.transpose())
sqrtYYT = U_Y.dot(np.diag(s_Y)).dot(U_Y.transpose())
print("Test YYT = U S^2 U^T :",np.linalg.norm(YYT - Y.dot(Y.transpose()),1)/np.linalg.norm(Y.dot(Y.transpose()),1),'(should be 0)')
print("Test square root of YYT :",np.linalg.norm(sqrtYYT.dot(sqrtYYT) - YYT,1)/np.linalg.norm(YYT,1),'(should be 0)')

A_tilde = M.dot(A).dot(sqrtYYT)
U_SVD,s_SVD,V_SVD = np.linalg.svd(A_tilde,full_matrices=True)
print("Test SVD :",np.linalg.norm((U_SVD.transpose().dot(U_SVD)) - np.eye(d),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((U_SVD.dot(U_SVD.transpose())) - np.eye(d),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((V_SVD.transpose().dot(V_SVD)) - np.eye(d),1),'(should be 0)')
print("Test SVD :",np.linalg.norm((V_SVD.dot(V_SVD.transpose())) - np.eye(d),1),'(should be 0)')

print("Test matrix inversion :",np.linalg.norm(np.linalg.inv(sqrtYYT).dot(sqrtYYT) - np.eye(d),1),'(should be 0)')
U,s,V = M.dot(U_SVD), s_SVD, np.linalg.inv(sqrtYYT).dot(V_SVD.transpose())
print("Test result :",np.linalg.norm(V.dot(YYT).dot(V.transpose()) - np.eye(d),1),'(should be 0)')
print("Test result :",np.linalg.norm(U.dot(U.transpose()) - np.eye(d),1),'(should be 0)')

Test SVD : 7.22647799021e-07 (should be 0)
Test SVD : 6.69716428092e-07 (should be 0)
Test SVD : 9.58843133578e-06 (should be 0)
Test SVD : 9.56596340984e-06 (should be 0)
Test YYT = U S^2 U^T : 1.09314e-06 (should be 0)
Test square root of YYT : 7.8858e-08 (should be 0)
Test SVD : 9.05843122409e-15 (should be 0)
Test SVD : 1.01250828739e-14 (should be 0)
Test SVD : 9.9805172197e-15 (should be 0)
Test SVD : 8.81492280428e-15 (should be 0)
Test matrix inversion : 13400.5488281 (should be 0)
Test result : 5.17204720354e+26 (should be 0)
Test result : 1.01250828739e-14 (should be 0)


In [156]:
V = np.linalg.inv(sqrtYYT).dot(V_SVD.transpose())
np.linalg.norm(V.dot(YYT).dot(V.transpose()) - np.eye(d),1)

9.9991381749167597e+25